<a href="https://colab.research.google.com/github/devinbook/Filipino-reddit-profanity/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers torch scikit-learn datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/combined_features_with_custom_vocab_and_full_vocab (2).csv")  # Replace with your file path

# Rename target column to 'labels' (required for Hugging Face Trainer)
df = df.rename(columns={"target": "labels"})

# Inspect the dataset
print(df.head())


   aabot  aabutin  aadjust  aalaga  aalala  aalis  aanak  aanhin  aanounce  \
0    0.0      0.0      0.0     0.0     0.0    0.0    0.0     0.0       0.0   
1    0.0      0.0      0.0     0.0     0.0    0.0    0.0     0.0       0.0   
2    0.0      0.0      0.0     0.0     0.0    0.0    0.0     0.0       0.0   
3    0.0      0.0      0.0     0.0     0.0    0.0    0.0     0.0       0.0   
4    0.0      0.0      0.0     0.0     0.0    0.0    0.0     0.0       0.0   

   aantay  ...  zero  zoom  zuckerberg  num_chars  num_words  num_sentences  \
0     0.0  ...   0.0   0.0         0.0        113         19              0   
1     0.0  ...   0.0   0.0         0.0         64          9              0   
2     0.0  ...   0.0   0.0         0.0         79         16              0   
3     0.0  ...   0.0   0.0         0.0         43          8              0   
4     0.0  ...   0.0   0.0         0.0         81         14              0   

   contains_profanity                                   

In [3]:
from datasets import Dataset

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split the dataset into training and testing sets
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


In [4]:
# Check for null values
print(df['Text'].isnull().sum())

# Inspect the dataset for any anomalies
print(df['Text'].head())

# Drop rows with missing Text values
df = df.dropna(subset=['Text'])

# Optionally reset the index
df = df.reset_index(drop=True)

1
0    kakausapin nakakapagod makipag usap mga tupa n...
1    pangalawang tahanan hinubog tayo nang husto ph...
2    pa pagka block fb ig naman jusq koya gago ka b...
3          would look think bobo ko talaga paano board
4    sinabi lang sana nilang work free trial para d...
Name: Text, dtype: object


In [5]:
# Check for null values
print(df['Text'].isnull().sum())

0


In [6]:
df['Text'] = df['Text'].astype(str)

In [7]:
from datasets import Dataset

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split into train and test datasets
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [8]:
from transformers import BertTokenizer

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Define a tokenization function
def tokenize_function(example):
    return tokenizer(
        example["Text"],  # Ensure this references the correct column
        truncation=True,
        padding=True,
        max_length=128  # Adjust as necessary
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [9]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/13616 [00:00<?, ? examples/s]

Map:   0%|          | 0/3405 [00:00<?, ? examples/s]

In [10]:
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


In [11]:
print(tokenized_train[0])  # View the first example in the training set


{'labels': tensor(2), 'input_ids': tensor([  101, 38008, 11252, 77443, 10133, 11252, 11264, 28659, 10133, 11387,
        27803, 30526, 14399, 10116, 64985, 10376, 13987, 15772,   173, 10237,
        59065, 63001, 15217, 96968, 11252, 11538, 20954, 10184, 10148, 11645,
        22821, 10184,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [12]:
from transformers import BertForSequenceClassification

# Load BERT model with a classification head (adjust for 4 classes: High, Mild, Moderate, Non-profane)
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=4)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',             # Directory to save the model checkpoints
    evaluation_strategy="epoch",       # Evaluate at the end of each epoch
    save_strategy="epoch",             # Save the model after each epoch
    learning_rate=2e-5,                # Learning rate
    per_device_train_batch_size=16,    # Batch size for training
    per_device_eval_batch_size=16,     # Batch size for evaluation
    num_train_epochs=5,                # Number of epochs
    weight_decay=0.01,                 # Weight decay for regularization
    logging_dir='./logs',              # Directory to save logs
    logging_steps=10,                  # Log every 10 steps
    load_best_model_at_end=True,       # Load the best model at the end of training
    metric_for_best_model="f1",        # Metric to determine the best model
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


<ipython-input-15-eb2e7713dc9b>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.085800,0.052584,0.991483,0.991487,0.991483,0.991479
2,0.001400,0.037320,0.994420,0.994447,0.994420,0.994414
3,0.074800,0.032683,0.994420,0.994428,0.994420,0.994418
4,0.002500,0.038135,0.995007,0.995026,0.995007,0.995005
5,0.000600,0.038467,0.995301,0.995317,0.995301,0.995300


TrainOutput(global_step=4255, training_loss=0.07034882917249473, metrics={'train_runtime': 2255.0819, 'train_samples_per_second': 30.19, 'train_steps_per_second': 1.887, 'total_flos': 4478230577233920.0, 'train_loss': 0.07034882917249473, 'epoch': 5.0})

In [18]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.0384671613574028, 'eval_accuracy': 0.9953010279001469, 'eval_precision': 0.9953172866106715, 'eval_recall': 0.9953010279001469, 'eval_f1': 0.9952995524993282, 'eval_runtime': 30.8656, 'eval_samples_per_second': 110.317, 'eval_steps_per_second': 6.901, 'epoch': 5.0}


In [19]:
# Save the model and tokenizer
model.save_pretrained('./Bert_profanitydetection')
tokenizer.save_pretrained('./Bert_profanitydetection')

print("Model and tokenizer saved as 'Bert_profanitydetection'")


Model and tokenizer saved as 'Bert_profanitydetection'


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
# Define the save path in Google Drive
save_path = '/content/drive/My Drive/Saved_Models/Bert_profanitydetection'

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to: {save_path}")


Model and tokenizer saved to: /content/drive/My Drive/Saved_Models/Bert_profanitydetection
